In [1]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


# Packages

In [2]:
import os
import torch
import pandas as pd
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence


# Data Preprocessing

In [3]:
class TransliterationDataset(Dataset):
    def __init__(self, file_path, latin_vocab=None, devanagari_vocab=None, max_len=50):

        # Read the file and check the number of columns
        self.data = pd.read_csv(file_path, sep='\t', header=None)
        
        # Adjust based on the number of columns in the data
        if self.data.shape[1] == 2:
            self.data.columns = ['devanagari', 'latin']
        elif self.data.shape[1] == 3:
            # Based on the screenshot, format is: [Devanagari, Latin, Frequency]
            self.data.columns = ['devanagari', 'latin', 'frequency']
        else:
            # Try to infer based on first row
            print(f"Warning: Unexpected number of columns ({self.data.shape[1]}). Inspecting first row:")
            print(self.data.iloc[0].tolist())
            # Default naming
            self.data.columns = [f'col{i}' for i in range(self.data.shape[1])]
        
        
        # Filter out rows with sequences longer than max_len
        self.data = self.data[
            (self.data['latin'].str.len() <= max_len) & 
            (self.data['devanagari'].str.len() <= max_len)
        ]
        
        # Define special tokens BEFORE creating vocabularies
        self.PAD_TOKEN = '<PAD>'
        self.SOS_TOKEN = '<SOS>'
        self.EOS_TOKEN = '<EOS>'
        
        self.PAD_IDX = 0
        self.SOS_IDX = 1
        self.EOS_IDX = 2
        
        # Create or use provided vocabularies
        if latin_vocab is None:
            # Create a new vocabulary
            char_to_idx = self._create_vocab(self.data['latin'])
            # Add special tokens
            self.latin_vocab = {
                self.PAD_TOKEN: self.PAD_IDX,
                self.SOS_TOKEN: self.SOS_IDX,
                self.EOS_TOKEN: self.EOS_IDX
            }
            # Add character tokens with indices starting after special tokens
            for char, idx in char_to_idx.items():
                self.latin_vocab[char] = idx + 3
        else:
            # Use the provided vocabulary
            self.latin_vocab = latin_vocab
            
        if devanagari_vocab is None:
            # Create a new vocabulary
            char_to_idx = self._create_vocab(self.data['devanagari'])
            # Add special tokens
            self.devanagari_vocab = {
                self.PAD_TOKEN: self.PAD_IDX,
                self.SOS_TOKEN: self.SOS_IDX,
                self.EOS_TOKEN: self.EOS_IDX
            }
            # Add character tokens with indices starting after special tokens
            for char, idx in char_to_idx.items():
                self.devanagari_vocab[char] = idx + 3
        else:
            # Use the provided vocabulary
            self.devanagari_vocab = devanagari_vocab
            
        # Create reverse mappings (index to character)
        self.latin_idx2char = {idx: char for char, idx in self.latin_vocab.items()}
        self.devanagari_idx2char = {idx: char for char, idx in self.devanagari_vocab.items()}
        
        # Print vocabulary sizes and special token information
        print(f"Latin vocabulary size: {len(self.latin_vocab)}")
        print(f"Devanagari vocabulary size: {len(self.devanagari_vocab)}")
        print(f"Special tokens: PAD={self.PAD_TOKEN} (idx={self.PAD_IDX}), "
              f"SOS={self.SOS_TOKEN} (idx={self.SOS_IDX}), "
              f"EOS={self.EOS_TOKEN} (idx={self.EOS_IDX})")
    
    def _create_vocab(self, series):
        """Create vocabulary from series of strings"""
        chars = set()
        for s in series:
            chars.update(s)
        return {char: idx for idx, char in enumerate(sorted(chars))}
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        latin_word = self.data.iloc[idx]['latin']
        devanagari_word = self.data.iloc[idx]['devanagari']
        
        # Convert to indices
        latin_indices = [self.latin_vocab[char] for char in latin_word]
        latin_indices = [self.SOS_IDX] + latin_indices + [self.EOS_IDX]
        
        devanagari_indices = [self.devanagari_vocab[char] for char in devanagari_word]
        devanagari_indices = [self.SOS_IDX] + devanagari_indices + [self.EOS_IDX]
        
        return {
            'latin': torch.tensor(latin_indices),
            'devanagari': torch.tensor(devanagari_indices),
            'latin_len': len(latin_indices),
            'devanagari_len': len(devanagari_indices),
            'latin_text': latin_word,
            'devanagari_text': devanagari_word
        }

# Data Loading

In [5]:
def collate_fn(batch):
    """Custom collate function for padding sequences in batch"""
    latin_seqs = [item['latin'] for item in batch]
    devanagari_seqs = [item['devanagari'] for item in batch]
    latin_lens = torch.tensor([item['latin_len'] for item in batch])
    devanagari_lens = torch.tensor([item['devanagari_len'] for item in batch])

    # Pad sequences
    latin_padded = pad_sequence(latin_seqs, batch_first=True, padding_value=0)
    devanagari_padded = pad_sequence(devanagari_seqs, batch_first=True, padding_value=0)

    latin_texts = [item['latin_text'] for item in batch]
    devanagari_texts = [item['devanagari_text'] for item in batch]

    return {
        'latin': latin_padded,
        'devanagari': devanagari_padded,
        'latin_len': latin_lens,
        'devanagari_len': devanagari_lens,
        'latin_text': latin_texts,
        'devanagari_text': devanagari_texts
    }

def get_dataloaders(train_path, val_path, test_path, batch_size=32, max_len=50):

    # Create train dataset and get vocabularies
    train_dataset = TransliterationDataset(train_path, max_len=max_len)

    # Create validation and test datasets with train vocabularies
    val_dataset = TransliterationDataset(
        val_path,
        latin_vocab=train_dataset.latin_vocab,
        devanagari_vocab=train_dataset.devanagari_vocab,
        max_len=max_len
    )

    test_dataset = TransliterationDataset(
        test_path,
        latin_vocab=train_dataset.latin_vocab,
        devanagari_vocab=train_dataset.devanagari_vocab,
        max_len=max_len
    )

    # Create dataloaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=collate_fn
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=collate_fn
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=collate_fn
    )

    return {
        'train_loader': train_loader,
        'val_loader': val_loader,
        'test_loader': test_loader,
        'latin_vocab': train_dataset.latin_vocab,
        'devanagari_vocab': train_dataset.devanagari_vocab,
        'latin_idx2char': train_dataset.latin_idx2char,
        'devanagari_idx2char': train_dataset.devanagari_idx2char
    }

# Encoder

In [6]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hidden_dim, n_layers, cell_type='GRU', dropout=0.0):

        super().__init__()

        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.cell_type = cell_type

        # Character embedding layer
        self.embedding = nn.Embedding(input_dim, emb_dim)

        # RNN layer
        if cell_type == 'RNN':
            self.rnn = nn.RNN(emb_dim, hidden_dim, n_layers, dropout=dropout if n_layers > 1 else 0, batch_first=True)
        elif cell_type == 'LSTM':
            self.rnn = nn.LSTM(emb_dim, hidden_dim, n_layers, dropout=dropout if n_layers > 1 else 0, batch_first=True)
        elif cell_type == 'GRU':
            self.rnn = nn.GRU(emb_dim, hidden_dim, n_layers, dropout=dropout if n_layers > 1 else 0, batch_first=True)
        else:
            raise ValueError(f"Unsupported RNN cell type: {cell_type}")

        self.dropout = nn.Dropout(dropout)

    def forward(self, src, src_len):

        # Convert to embeddings
        embedded = self.dropout(self.embedding(src))  # [batch_size, seq_len, emb_dim]

        # Pack sequences to handle variable lengths efficiently
        packed_embedded = nn.utils.rnn.pack_padded_sequence(
            embedded, src_len.cpu(), batch_first=True, enforce_sorted=False
        )

        # Pass through RNN
        if self.cell_type == 'LSTM':
            packed_outputs, (hidden, cell) = self.rnn(packed_embedded)
            # Unpack sequence
            outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs, batch_first=True)
            return outputs, (hidden, cell)
        else:
            packed_outputs, hidden = self.rnn(packed_embedded)
            # Unpack sequence
            outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs, batch_first=True)
            return outputs, hidden

# Decoder

In [7]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hidden_dim, n_layers, cell_type='GRU', dropout=0.0):
        super().__init__()

        self.output_dim = output_dim
        self.emb_dim = emb_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.cell_type = cell_type

        # Character embedding layer
        self.embedding = nn.Embedding(output_dim, emb_dim)

        # RNN layer
        if cell_type == 'RNN':
            self.rnn = nn.RNN(emb_dim, hidden_dim, n_layers, dropout=dropout if n_layers > 1 else 0, batch_first=True)
        elif cell_type == 'LSTM':
            self.rnn = nn.LSTM(emb_dim, hidden_dim, n_layers, dropout=dropout if n_layers > 1 else 0, batch_first=True)
        elif cell_type == 'GRU':
            self.rnn = nn.GRU(emb_dim, hidden_dim, n_layers, dropout=dropout if n_layers > 1 else 0, batch_first=True)
        else:
            raise ValueError(f"Unsupported RNN cell type: {cell_type}")

        # Linear layer to produce output probabilities
        self.fc_out = nn.Linear(hidden_dim, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden):

        # Convert to embeddings and apply dropout
        input = input.unsqueeze(1)  # Add sequence dimension [batch_size, 1]
        embedded = self.dropout(self.embedding(input))  # [batch_size, 1, emb_dim]

        # Pass through RNN
        if self.cell_type == 'LSTM':
            output, (hidden, cell) = self.rnn(embedded, hidden)
        else:
            output, hidden = self.rnn(embedded, hidden)

        # Get prediction
        prediction = self.fc_out(output.squeeze(1))  # [batch_size, output_dim]

        # Return prediction and hidden state
        if self.cell_type == 'LSTM':
            return prediction, (hidden, cell)
        else:
            return prediction, hidden

# Encoder-Decoder Adapter

In [8]:
class EncoderDecoderAdapter(nn.Module):
    """
    Adapter module to connect encoder and decoder with different layer counts
    """
    def __init__(self, encoder_layers, decoder_layers, hidden_dim, cell_type):
        super().__init__()
        self.encoder_layers = encoder_layers
        self.decoder_layers = decoder_layers
        self.hidden_dim = hidden_dim
        self.cell_type = cell_type

        # Map features: fully-connected network for each decoder layer
        self.hidden_adapters = nn.ModuleList([
            nn.Linear(encoder_layers * hidden_dim, hidden_dim)
            for _ in range(decoder_layers)
        ])

        # Cell state adapters for LSTM
        if cell_type == 'LSTM':
            self.cell_adapters = nn.ModuleList([
                nn.Linear(encoder_layers * hidden_dim, hidden_dim)
                for _ in range(decoder_layers)
            ])

    def forward(self, encoder_hidden):
        
        if self.cell_type == 'LSTM':
            hidden, cell = encoder_hidden
            batch_size = hidden.size(1)

            # Reshape to [batch_size, encoder_layers * hidden_dim]
            hidden_flat = hidden.permute(1, 0, 2).contiguous().view(batch_size, -1)
            cell_flat = cell.permute(1, 0, 2).contiguous().view(batch_size, -1)

            # Create new hidden and cell states for decoder
            decoder_hidden = []
            decoder_cell = []

            # Apply adapter networks for each decoder layer
            for i in range(self.decoder_layers):
                decoder_hidden.append(self.hidden_adapters[i](hidden_flat))
                decoder_cell.append(self.cell_adapters[i](cell_flat))

            # Stack and reshape to [decoder_layers, batch_size, hidden_dim]
            decoder_hidden = torch.stack(decoder_hidden, dim=0)
            decoder_cell = torch.stack(decoder_cell, dim=0)

            return (decoder_hidden, decoder_cell)

        else:  # RNN or GRU
            batch_size = encoder_hidden.size(1)

            # Reshape to [batch_size, encoder_layers * hidden_dim]
            hidden_flat = encoder_hidden.permute(1, 0, 2).contiguous().view(batch_size, -1)

            # Create new hidden state for decoder
            decoder_hidden = []

            # Apply adapter networks for each decoder layer
            for i in range(self.decoder_layers):
                decoder_hidden.append(self.hidden_adapters[i](hidden_flat))

            # Stack and reshape to [decoder_layers, batch_size, hidden_dim]
            decoder_hidden = torch.stack(decoder_hidden, dim=0)

            return decoder_hidden

# Seq2Seq Model

In [9]:
import random
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device, teacher_forcing_ratio=0.5):

        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        self.teacher_forcing_ratio = teacher_forcing_ratio

        # Check that hidden dimensions match
        assert encoder.hidden_dim == decoder.hidden_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"

        # Check that cell types match
        assert encoder.cell_type == decoder.cell_type, \
            "Encoder and decoder must use the same cell type!"

        # Create adapter if layer counts differ
        if encoder.n_layers != decoder.n_layers:
            print(f"Creating adapter from {encoder.n_layers} encoder layers to {decoder.n_layers} decoder layers")
            self.adapter = EncoderDecoderAdapter(
                encoder_layers=encoder.n_layers,
                decoder_layers=decoder.n_layers,
                hidden_dim=encoder.hidden_dim,
                cell_type=encoder.cell_type
            )
        else:
            self.adapter = None

    def forward(self, src, src_len, trg, teacher_forcing_ratio=None):

        batch_size = src.shape[0]
        trg_len = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim

        # Tensor to store decoder outputs
        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(self.device)

        # Encode source sequence
        if self.encoder.cell_type == 'LSTM':
            encoder_outputs, (hidden, cell) = self.encoder(src, src_len)

            # Adapt encoder hidden state to decoder format if needed
            if self.adapter is not None:
                hidden, cell = self.adapter((hidden, cell))

            # Set initial decoder state
            decoder_hidden = (hidden, cell)
        else:
            encoder_outputs, hidden = self.encoder(src, src_len)

            # Adapt encoder hidden state to decoder format if needed
            if self.adapter is not None:
                hidden = self.adapter(hidden)

            # Set initial decoder state
            decoder_hidden = hidden

        # First input to the decoder is the <SOS> token
        input = trg[:, 0]  # Shape: [batch_size]

        # Use teacher forcing ratio from argument if provided, else use default
        if teacher_forcing_ratio is None:
            teacher_forcing_ratio = self.teacher_forcing_ratio

        # Decode one step at a time
        for t in range(1, trg_len):
            # Pass through decoder
            output, decoder_hidden = self.decoder(input, decoder_hidden)

            # Save to outputs tensor
            outputs[:, t, :] = output

            # Decide whether to use teacher forcing
            teacher_force = random.random() < teacher_forcing_ratio

            # Get the highest predicted token from our predictions
            top1 = output.argmax(1)

            # If teacher forcing, use actual next token as next input, else use predicted token
            input = trg[:, t] if teacher_force else top1

        return outputs

# Training & Evaluation (1 epoch)

In [10]:
import torch.optim as optim
import torch.nn.functional as F
import time
import math

def train(model, dataloader, optimizer, criterion, clip, device):
    
    #Training function for one epoch

    model.train()
    epoch_loss = 0

    for batch in dataloader:
        # Get data
        src = batch['latin'].to(device)
        trg = batch['devanagari'].to(device)
        src_len = batch['latin_len'].to(device)

        optimizer.zero_grad()

        # Forward pass
        output = model(src, src_len, trg)

        # Calculate loss
        # output shape: [batch_size, trg_len, output_dim]
        # trg shape: [batch_size, trg_len]
        # Ignore the <SOS> token (first token)
        output_dim = output.shape[-1]
        output = output[:, 1:].reshape(-1, output_dim)
        trg = trg[:, 1:].reshape(-1)

        loss = criterion(output, trg)

        # Backward pass
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        # Update parameters
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(dataloader)

def evaluate(model, dataloader, criterion, device):
    
    model.eval()
    epoch_loss = 0

    with torch.no_grad():
        for batch in dataloader:
            # Get data
            src = batch['latin'].to(device)
            trg = batch['devanagari'].to(device)
            src_len = batch['latin_len'].to(device)

            # Forward pass
            output = model(src, src_len, trg, 0)  # Turn off teacher forcing

            # Calculate loss
            output_dim = output.shape[-1]
            output = output[:, 1:].reshape(-1, output_dim)
            trg = trg[:, 1:].reshape(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(dataloader)

# Inference

In [11]:
def translate_with_beam(model, src, src_len, devanagari_idx2char, device, sos_idx=1, eos_idx=2, pad_idx=0, beam_size=5, max_len=50):
    """
    Translate a source sequence to target language using beam search
    """
    model.eval()
    batch_size = src.shape[0]
    translations = []

    with torch.no_grad():
        # Process each example in the batch separately
        for i in range(batch_size):
            # Get the individual source sequence
            src_seq = src[i:i+1]  # Keep batch dimension: [1, src_len]
            src_len_seq = src_len[i:i+1]  # [1]

            # Encode the source sequence
            if model.encoder.cell_type == 'LSTM':
                encoder_outputs, (hidden, cell) = model.encoder(src_seq, src_len_seq)
                
                # Apply adapter if encoder and decoder have different layer counts
                if model.adapter is not None:
                    hidden, cell = model.adapter((hidden, cell))
                
                # Set initial decoder state
                hidden_states = (hidden, cell)
            else:
                encoder_outputs, hidden = model.encoder(src_seq, src_len_seq)
                
                # Apply adapter if encoder and decoder have different layer counts
                if model.adapter is not None:
                    hidden = model.adapter(hidden)
                    
                # Set initial decoder state
                hidden_states = hidden

            # Initialize beam search
            # Each beam will contain: (sequence, score, hidden_states, completed_flag)
            beams = []

            # Start with SOS token
            beams.append(([sos_idx], 0.0, hidden_states, False))

            # Generate sequence token by token
            for t in range(max_len - 1):  # -1 because we already added SOS
                new_beams = []

                # Flag to check if all beams have completed
                all_beams_completed = True

                # Expand each current beam
                for seq, score, states, completed in beams:
                    if completed:
                        # If this beam is already complete, keep it
                        new_beams.append((seq, score, states, completed))
                        continue

                    # At least one beam is not completed
                    all_beams_completed = False

                    # Get the last token in the sequence
                    last_token = torch.tensor([seq[-1]], device=device)

                    # Forward pass through the decoder
                    output, new_states = model.decoder(last_token, states)

                    # Get probabilities
                    probs = torch.nn.functional.log_softmax(output, dim=1)

                    # Get top-k next tokens
                    topk_probs, topk_idx = probs.topk(beam_size)

                    # Create new beams
                    for j in range(beam_size):
                        next_token = topk_idx[0][j].item()
                        next_score = score + topk_probs[0][j].item()
                        next_seq = seq + [next_token]
                        next_completed = (next_token == eos_idx)

                        new_beams.append((next_seq, next_score, new_states, next_completed))

                # Update beams: select top-k beams
                # Normalize by length to avoid bias toward shorter sequences
                beams = sorted(new_beams, key=lambda x: x[1] / len(x[0]), reverse=True)[:beam_size]

                # If all beams have completed, break
                if all_beams_completed:
                    break

            # Select the best beam
            best_seq = beams[0][0]

            # Convert to characters
            chars = []
            for token in best_seq:
                if token == eos_idx:
                    break
                if token != sos_idx and token != pad_idx:
                    chars.append(devanagari_idx2char[token])

            translations.append(''.join(chars))

    return translations

In [12]:
def translate(model, src, src_len, devanagari_idx2char, device, sos_idx=1, eos_idx=2, pad_idx=0, max_len=50):
    """
    Translate a source sequence to target language using greedy decoding
    """
    model.eval()
    batch_size = src.shape[0]
    translations = []

    with torch.no_grad():
        # Encode the source sequence
        if model.encoder.cell_type == 'LSTM':
            encoder_outputs, (hidden, cell) = model.encoder(src, src_len)
            
            # Apply adapter if encoder and decoder have different layer counts
            if model.adapter is not None:
                hidden, cell = model.adapter((hidden, cell))
                
            # Set initial decoder state
            hidden_states = (hidden, cell)
        else:
            encoder_outputs, hidden = model.encoder(src, src_len)
            
            # Apply adapter if encoder and decoder have different layer counts
            if model.adapter is not None:
                hidden = model.adapter(hidden)
                
            # Set initial decoder state
            hidden_states = hidden

        # Start with < SOS > token for each example in batch
        input = torch.tensor([sos_idx] * batch_size, device=device)

        # Initialize result sequences
        result_sequences = torch.full((batch_size, max_len), pad_idx, dtype=torch.long, device=device)
        result_sequences[:, 0] = input

        # Track which sequences have ended
        ended_sequences = torch.zeros(batch_size, dtype=torch.bool, device=device)

        # Generate one character at a time
        for t in range(1, max_len):
            # Pass through decoder
            output, hidden_states = model.decoder(input, hidden_states)

            # Get the predicted token
            pred_token = output.argmax(1)

            # Save the predicted token
            result_sequences[:, t] = pred_token

            # Mark sequences that have ended (predicted <EOS>)
            ended_sequences = ended_sequences | (pred_token == eos_idx)

            # Stop if all sequences have ended
            if ended_sequences.all():
                break

            # Next input is the predicted token
            input = pred_token

    # Convert indices to characters
    for i in range(batch_size):
        seq = result_sequences[i].cpu().numpy()
        # Convert to string, stopping at <EOS> token
        chars = []
        for idx in seq:
            if idx == eos_idx:
                break
            if idx != sos_idx and idx != pad_idx:
                chars.append(devanagari_idx2char[idx.item()])

        translations.append(''.join(chars))

    return translations

# Calculate Accuracy

In [13]:
def calculate_accuracy(model, dataloader, devanagari_idx2char, device, beam_size=1, sos_idx=1, eos_idx=2, pad_idx=0):
    
    #Calculate accuracy on a dataset using either greedy or beam search

    model.eval()

    correct = 0
    total = 0
    predictions = []
    targets = []

    with torch.no_grad():
        for batch in dataloader:
            src = batch['latin'].to(device)
            src_len = batch['latin_len'].to(device)
            trg_texts = batch['devanagari_text']

            # Use either greedy or beam search
            if beam_size <= 1:
                # Use greedy search
                translations = translate(
                    model, src, src_len, devanagari_idx2char, device,
                    sos_idx=sos_idx, eos_idx=eos_idx, pad_idx=pad_idx
                )
            else:
                # Use beam search
                translations = translate_with_beam(
                    model, src, src_len, devanagari_idx2char, device,
                    sos_idx=sos_idx, eos_idx=eos_idx, pad_idx=pad_idx,
                    beam_size=beam_size
                )

            for pred, gold in zip(translations, trg_texts):
                predictions.append(pred)
                targets.append(gold)
                if pred == gold:
                    correct += 1
                total += 1

    return (correct / total), predictions, targets

# Train with wandb

In [30]:
import wandb

def train_model(config=None, run_name=None):

    if config is not None:
        config = wandb.config
        wandb.run.name = run_name
    # Otherwise, use default (Best Model)
    elif config is None:
        config = {
            'emb_dim': 256,
            'hidden_dim': 512,
            'encoder_layers': 2,
            'decoder_layers': 2,
            'cell_type': 'LSTM',
            'dropout': 0.2,
            'lr': 0.001,
            'batch_size': 128,
            'epochs': 10,
            'clip': 1.0,
            'teacher_forcing_ratio': 0.3,
            'beam_size': 3  # Start with greedy search (1) for stability
        }

    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Load data
    data = get_dataloaders(
        '/kaggle/input/dakshina/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv',
        '/kaggle/input/dakshina/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.dev.tsv',
        '/kaggle/input/dakshina/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.test.tsv',
        batch_size=config['batch_size']
    )

    train_loader = data['train_loader']
    val_loader = data['val_loader']
    test_loader = data['test_loader']

    latin_vocab = data['latin_vocab']
    devanagari_vocab = data['devanagari_vocab']
    latin_idx2char = data['latin_idx2char']
    devanagari_idx2char = data['devanagari_idx2char']
    
    # Get special token indices directly
    # Extract from the dataset to ensure consistency
    train_dataset = train_loader.dataset
    # Use default values if the attributes don't exist
    sos_idx = getattr(train_dataset, 'SOS_IDX', 1)
    eos_idx = getattr(train_dataset, 'EOS_IDX', 2) 
    pad_idx = getattr(train_dataset, 'PAD_IDX', 0)
    

    # Create model
    input_dim = len(latin_vocab)
    output_dim = len(devanagari_vocab)

    encoder = Encoder(
        input_dim=input_dim,
        emb_dim=config['emb_dim'],
        hidden_dim=config['hidden_dim'],
        n_layers=config['encoder_layers'],
        cell_type=config['cell_type'],
        dropout=config['dropout']
    )

    decoder = Decoder(
        output_dim=output_dim,
        emb_dim=config['emb_dim'],
        hidden_dim=config['hidden_dim'],
        n_layers=config['decoder_layers'],
        cell_type=config['cell_type'],
        dropout=config['dropout']
    )

    model = Seq2Seq(
        encoder=encoder,
        decoder=decoder,
        device=device,
        teacher_forcing_ratio=config['teacher_forcing_ratio']
    )

    model = model.to(device)

    # Calculate number of parameters
    num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f'The model has {num_params:,} trainable parameters')

    # Initialize optimizer and criterion
    optimizer = optim.Adam(model.parameters(), lr=config['lr'])
    criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)  # Ignore padding index

    # Training loop
    best_val_loss = float('inf')

    for epoch in range(config['epochs']):
        start_time = time.time()

        train_loss = train(model, train_loader, optimizer, criterion, config['clip'], device)
        val_loss = evaluate(model, val_loader, criterion, device)

        end_time = time.time()
        epoch_mins, epoch_secs = divmod(end_time - start_time, 60)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'best-model.pt')

        print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs:.2f}s')
        print(f'\tTrain Loss: {train_loss:.3f}')
        print(f'\t Val. Loss: {val_loss:.3f}')

        # Validation accuracy with updated function
        val_accuracy, _, _ = calculate_accuracy(
            model, val_loader, devanagari_idx2char, device, 
            beam_size=1,  # Use greedy search during training for speed
            sos_idx=sos_idx, eos_idx=eos_idx, pad_idx=pad_idx
        )
        print(f'\t Val. Accuracy: {val_accuracy:.3f}')

        # Log to wandb
        if wandb.run is not None:
            wandb.log({
                "train_loss": train_loss,
                "val_loss": val_loss,
                "val_accuracy": val_accuracy,
                "epoch": epoch
            })

    # Load the best model
    model.load_state_dict(torch.load('best-model.pt'))

    # Calculate test accuracy with beam search
    test_accuracy, predictions, targets = calculate_accuracy(
        model, test_loader, devanagari_idx2char, device, 
        beam_size=config['beam_size'],  # Use beam search for final evaluation
        sos_idx=sos_idx, eos_idx=eos_idx, pad_idx=pad_idx
    )
    print(f'Test Accuracy (beam size={config["beam_size"]}): {test_accuracy:.3f}')

    # if wandb.run is not None:
    #     wandb.log({"test_accuracy": test_accuracy})

    #Save predictions for analysis
    latin_texts = []
    devanagari_texts = []

    for batch in test_loader:
        latin_texts.extend(batch['latin_text'])
        devanagari_texts.extend(batch['devanagari_text'])

    # Create DataFrame with all 4 columns
    results = pd.DataFrame({
        'latin': latin_texts[:len(predictions)],
        'true': devanagari_texts[:len(predictions)],
        'predicted': predictions,
        'accuracy': [1 if pred == target else 0 for pred, target in zip(predictions, devanagari_texts[:len(predictions)])]
    })

    
    results.to_csv('./test_vanilla.csv', index=False)
    

    return model  #, test_accuracy

# Sweep Configuration

In [16]:
sweep_config = {
        'method': 'bayes',  # Bayesian optimization
        'metric': {
            'name': 'val_accuracy',
            'goal': 'maximize'
        },
        'parameters': {
            'emb_dim': {
                'values': [16, 32, 64, 128, 256]
            },
            'hidden_dim': {
                'values': [32, 64, 128, 256, 512]
            },
            'encoder_layers': {
                'values': [1, 2, 3]
            },
            'decoder_layers': {
                'values': [1, 2, 3]
            },
            'cell_type': {
                'values': ['RNN', 'GRU', 'LSTM']
            },
            'dropout': {
                'values': [0.1, 0.2, 0.3,]
            },
            'lr': {
                'values': [0.0001, 0.001, 0.01]
            },
            'batch_size': {
                'values': [32, 64, 128]
            },
            'clip': {
                'values': [0.1, 1.0, 5.0]
            },
            'teacher_forcing_ratio': {
                'values': [0.3, 0.5, 0.7]
            },
            'beam_size': {
                'values': [1, 3, 5]  # 1 is equivalent to greedy search
            },
            'epochs': {
                'values': [2]
            }
        }
}

def run_sweep():
        run = wandb.init()
        config = wandb.config
        run_name = (
        f"{config.cell_type}_"
        f"enc_{config.encoder_layers}_dec_{config.decoder_layers}_"
        f"hdim_{config.hidden_dim}_"
        f"emb_{config.emb_dim}_"
        f"bs_{config.batch_size}_"
        f"drop_{config.dropout}_"
        f"beam_{config.beam_size}_"
        f"lr_{config.lr}_"
        f"clip_{config.clip}_"
        f"tf_{config.teacher_forcing_ratio}_"
        f"epoch_{config.epochs}"
        )
        train_model(config=wandb.config, run_name=run_name)  # wandb.config is automatically passed


# Local Execution

In [31]:
model=train_model(config=None, run_name=None)

/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less_equal
  return op(a, b)


Latin vocabulary size: 29
Devanagari vocabulary size: 66
Special tokens: PAD=<PAD> (idx=0), SOS=<SOS> (idx=1), EOS=<EOS> (idx=2)
Latin vocabulary size: 29
Devanagari vocabulary size: 66
Special tokens: PAD=<PAD> (idx=0), SOS=<SOS> (idx=1), EOS=<EOS> (idx=2)
Latin vocabulary size: 29
Devanagari vocabulary size: 66
Special tokens: PAD=<PAD> (idx=0), SOS=<SOS> (idx=1), EOS=<EOS> (idx=2)
The model has 7,414,594 trainable parameters
Epoch: 01 | Time: 0.0m 17.83s
	Train Loss: 1.703
	 Val. Loss: 1.132
	 Val. Accuracy: 0.207
Epoch: 02 | Time: 0.0m 17.92s
	Train Loss: 0.831
	 Val. Loss: 0.964
	 Val. Accuracy: 0.304
Epoch: 03 | Time: 0.0m 17.88s
	Train Loss: 0.632
	 Val. Loss: 0.913
	 Val. Accuracy: 0.345
Epoch: 04 | Time: 0.0m 18.00s
	Train Loss: 0.498
	 Val. Loss: 0.935
	 Val. Accuracy: 0.351
Epoch: 05 | Time: 0.0m 18.01s
	Train Loss: 0.411
	 Val. Loss: 0.948
	 Val. Accuracy: 0.353
Epoch: 06 | Time: 0.0m 17.98s
	Train Loss: 0.342
	 Val. Loss: 0.935
	 Val. Accuracy: 0.374
Epoch: 07 | Time: 0.0m

# Wandb Execution

In [ ]:

wandb.login()
sweep_id = wandb.sweep(sweep_config,project="DA6401-A3", entity="cs24m033-iit-madras")
wandb.agent(sweep_id,function=run_sweep, count = 22 )

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: r9ulhvpq
Sweep URL: https://wandb.ai/cs24m033-iit-madras/DA6401-A3/sweeps/r9ulhvpq


wandb: Agent Starting Run: 5eem3lkq with config:
wandb: 	batch_size: 128
wandb: 	beam_size: 1
wandb: 	cell_type: LSTM
wandb: 	clip: 5
wandb: 	decoder_layers: 3
wandb: 	dropout: 0.3
wandb: 	emb_dim: 256
wandb: 	encoder_layers: 2
wandb: 	epochs: 15
wandb: 	hidden_dim: 512
wandb: 	lr: 0.001
wandb: 	teacher_forcing_ratio: 0.3


/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less_equal
  return op(a, b)


Latin vocabulary size: 29
Devanagari vocabulary size: 66
Special tokens: PAD=<PAD> (idx=0), SOS=<SOS> (idx=1), EOS=<EOS> (idx=2)
Latin vocabulary size: 29
Devanagari vocabulary size: 66
Special tokens: PAD=<PAD> (idx=0), SOS=<SOS> (idx=1), EOS=<EOS> (idx=2)
Latin vocabulary size: 29
Devanagari vocabulary size: 66
Special tokens: PAD=<PAD> (idx=0), SOS=<SOS> (idx=1), EOS=<EOS> (idx=2)
Creating adapter from 2 encoder layers to 3 decoder layers
The model has 12,664,642 trainable parameters
Epoch: 01 | Time: 0.0m 21.17s
	Train Loss: 2.009
	 Val. Loss: 1.261
	 Val. Accuracy: 0.137
Epoch: 02 | Time: 0.0m 21.03s
	Train Loss: 0.977
	 Val. Loss: 1.013
	 Val. Accuracy: 0.265
Epoch: 03 | Time: 0.0m 21.12s
	Train Loss: 0.733
	 Val. Loss: 0.937
	 Val. Accuracy: 0.327
Epoch: 04 | Time: 0.0m 20.92s
	Train Loss: 0.585
	 Val. Loss: 0.944
	 Val. Accuracy: 0.333
Epoch: 05 | Time: 0.0m 20.82s
	Train Loss: 0.478
	 Val. Loss: 0.946
	 Val. Accuracy: 0.354
Epoch: 06 | Time: 0.0m 21.12s
	Train Loss: 0.392
	 Va

...

epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train_loss,█▅▄▃▃▃▂▂▂▂▂▁▁▁▁
val_accuracy,▁▂▄▅▅▆▆▇▇▇▇████
val_loss,█▅▃▃▂▂▂▁▁▁▁▁▁▁▁
epoch,14
train_loss,0.48379
val_accuracy,0.352
val_loss,0.97644


wandb: Agent Starting Run: 03u4qjpd with config:
wandb: 	batch_size: 64
wandb: 	beam_size: 1
wandb: 	cell_type: GRU
wandb: 	clip: 5
wandb: 	decoder_layers: 2
wandb: 	dropout: 0.3
wandb: 	emb_dim: 256
wandb: 	encoder_layers: 3
wandb: 	epochs: 15
wandb: 	hidden_dim: 512
wandb: 	lr: 0.0001
wandb: 	teacher_forcing_ratio: 0.5


/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less_equal
  return op(a, b)


Latin vocabulary size: 29
Devanagari vocabulary size: 66
Special tokens: PAD=<PAD> (idx=0), SOS=<SOS> (idx=1), EOS=<EOS> (idx=2)
Latin vocabulary size: 29
Devanagari vocabulary size: 66
Special tokens: PAD=<PAD> (idx=0), SOS=<SOS> (idx=1), EOS=<EOS> (idx=2)
Latin vocabulary size: 29
Devanagari vocabulary size: 66
Special tokens: PAD=<PAD> (idx=0), SOS=<SOS> (idx=1), EOS=<EOS> (idx=2)
Creating adapter from 3 encoder layers to 2 decoder layers
The model has 8,725,314 trainable parameters
Epoch: 01 | Time: 0.0m 28.38s
	Train Loss: 2.213
	 Val. Loss: 1.653
	 Val. Accuracy: 0.076
Epoch: 02 | Time: 0.0m 28.07s
	Train Loss: 1.282
	 Val. Loss: 1.341
	 Val. Accuracy: 0.165
Epoch: 03 | Time: 0.0m 28.07s
	Train Loss: 1.014
	 Val. Loss: 1.225
	 Val. Accuracy: 0.227
Epoch: 04 | Time: 0.0m 28.05s
	Train Loss: 0.871
	 Val. Loss: 1.140
	 Val. Accuracy: 0.271
Epoch: 05 | Time: 0.0m 28.10s
	Train Loss: 0.766
	 Val. Loss: 1.104
	 Val. Accuracy: 0.296
Epoch: 06 | Time: 0.0m 28.20s
	Train Loss: 0.694
	 Val

epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train_loss,█▅▄▃▃▂▂▂▂▂▁▁▁▁▁
val_accuracy,▁▃▅▆▆▇▇▇█▇██▇██
val_loss,█▄▃▂▂▁▁▁▁▁▁▁▂▁▂
epoch,14
train_loss,0.3369
val_accuracy,0.3637
val_loss,1.11681
